In [1]:
from datasets import load_dataset
dataset = load_dataset('parquet', data_files='data/train.parquet')

Generating train split: 0 examples [00:00, ? examples/s]

In [3]:
# Load the first 1000 rows of the dataset
train_dataset = dataset['train'].select(range(1000))
eval_dataset = dataset['train'].select(range(1000, 1200))
# test_dataset = dataset['test'].select(range(1000))

KeyError: 'test'

In [5]:
all_labels = {label for labels in dataset['train']['labels'] for label in labels}
num_labels = len(all_labels)

In [12]:
label2id = { index:label for index, label in enumerate(all_labels)}
id2label = { value:key for key,value in label2id.items()}

In [17]:
from transformers import BertForSequenceClassification, TrainingArguments, Trainer

model = BertForSequenceClassification.from_pretrained('huawei-noah/TinyBERT_General_4L_312D', num_labels=num_labels,                                             
 label2id=label2id,
 id2label=id2label)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at huawei-noah/TinyBERT_General_4L_312D and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

In [19]:
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

def compute_metrics(eval_pred):
    """Computes evaluation metrics for multi-label classification."""
    logits, labels = eval_pred
    predictions = (logits > 0.5).astype(int)

    f1 = f1_score(labels, predictions, average='macro')
    acc = accuracy_score(labels, predictions)
    precision = precision_score(labels, predictions, average='macro')
    recall = recall_score(labels, predictions, average='macro')

    return {
        'f1': f1,
        'accuracy': acc,
        'precision': precision,
        'recall': recall
    }

In [20]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics
)

In [21]:
trainer.train()

ValueError: too many dimensions 'str'